# Case Study: FIRE at 45

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/engineerinvestor/monteplan/blob/main/notebooks/04_case_study_fire.ipynb)

**Persona:** Alex, age 30, software engineer. Earns $12,000/month, saves aggressively ($60,000/year across accounts), and wants to retire at 45 with $3,500/month spending.

**Challenge:** A 45-year retirement (age 45-90) is much longer than a traditional 30-year retirement. Can Alex make it work?

In [ ]:
# Uncomment and run in Google Colab:
# !pip install -q "monteplan @ git+https://github.com/engineerinvestor/monteplan.git"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from monteplan.config.defaults import fire_plan, default_market, default_policies
from monteplan.config.schema import (
    SimulationConfig, PolicyBundle, SpendingPolicyConfig,
    GuardrailsConfig, VPWConfig, StressScenario, PlanConfig, AccountConfig,
)
from monteplan.core.engine import simulate

## Alex's Plan

In [ ]:
plan = fire_plan()
market = default_market()
policies = default_policies()
sim_config = SimulationConfig(n_paths=1000, seed=42)

print(f"Age: {plan.current_age} -> Retire: {plan.retirement_age} -> Horizon: {plan.end_age}")
print(f"Monthly spending: ${plan.monthly_spending:,.0f}")
print(f"Annual savings: ${sum(a.annual_contribution for a in plan.accounts):,.0f}")
print(f"Current portfolio: ${sum(a.balance for a in plan.accounts):,.0f}")

## Base Case Simulation

In [ ]:
result = simulate(plan, market, policies, sim_config)

print(f"Success probability: {result.success_probability:.1%}")
print(f"\nThis means about {result.success_probability*100:.0f} out of 100 simulated")
print(f"futures, Alex's portfolio lasts to age {plan.end_age}.")

# Fan chart
ts = result.wealth_time_series
ages = np.linspace(plan.current_age, plan.end_age, len(ts["p50"]))

fig, ax = plt.subplots(figsize=(10, 5))
ax.fill_between(ages, ts["p5"], ts["p95"], alpha=0.15, color="steelblue")
ax.fill_between(ages, ts["p25"], ts["p75"], alpha=0.3, color="steelblue")
ax.plot(ages, ts["p50"], color="steelblue", linewidth=2)
ax.axvline(plan.retirement_age, color="gray", linestyle="--", alpha=0.5, label="Retirement (45)")
ax.set_xlabel("Age")
ax.set_ylabel("Portfolio Value ($)")
ax.set_title(f"FIRE Plan: {result.success_probability:.1%} Success")
ax.legend()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"${x/1e6:.1f}M" if x >= 1e6 else f"${x/1e3:.0f}K"))
plt.tight_layout()
plt.show()

## What Would It Take?

Let's sweep key parameters to see what moves the needle:

In [ ]:
# Sweep monthly spending
spending_levels = [2500, 3000, 3500, 4000, 4500]
spending_results = []
for sp in spending_levels:
    p = plan.model_copy(update={"monthly_spending": sp})
    r = simulate(p, market, policies, sim_config)
    spending_results.append(r.success_probability)
    print(f"  ${sp:,.0f}/mo -> {r.success_probability:.1%}")

In [ ]:
# Sweep retirement age
ret_ages = [40, 42, 45, 48, 50]
ret_results = []
for ra in ret_ages:
    p = plan.model_copy(update={"retirement_age": ra})
    r = simulate(p, market, policies, sim_config)
    ret_results.append(r.success_probability)
    print(f"  Retire at {ra} -> {r.success_probability:.1%}")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(spending_levels, spending_results, "o-", color="steelblue", linewidth=2)
ax1.set_xlabel("Monthly Spending ($)")
ax1.set_ylabel("Success Probability")
ax1.set_title("Success vs Spending")
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x:.0%}"))
ax1.axhline(0.8, color="green", linestyle="--", alpha=0.5, label="80% target")
ax1.legend()

ax2.plot(ret_ages, ret_results, "o-", color="steelblue", linewidth=2)
ax2.set_xlabel("Retirement Age")
ax2.set_ylabel("Success Probability")
ax2.set_title("Success vs Retirement Age")
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x:.0%}"))
ax2.axhline(0.8, color="green", linestyle="--", alpha=0.5, label="80% target")
ax2.legend()

plt.tight_layout()
plt.show()

## Spending Policy Comparison for Long Retirements

For a 45-year retirement, adaptive spending policies shine:

In [ ]:
policy_configs = {
    "Constant Real": PolicyBundle(spending=SpendingPolicyConfig(policy_type="constant_real")),
    "Guardrails": PolicyBundle(
        spending=SpendingPolicyConfig(
            policy_type="guardrails",
            guardrails=GuardrailsConfig(initial_withdrawal_rate=0.04),
        ),
    ),
    "VPW": PolicyBundle(
        spending=SpendingPolicyConfig(
            policy_type="vpw",
            vpw=VPWConfig(min_rate=0.02, max_rate=0.12),
        ),
    ),
}

for name, pol in policy_configs.items():
    r = simulate(plan, market, pol, sim_config)
    print(f"{name:20s} -> {r.success_probability:.1%}")

## Stress Test: Market Crash at Age 44

In [ ]:
crash_sim = SimulationConfig(
    n_paths=1000, seed=42,
    stress_scenarios=[
        StressScenario(name="Crash", scenario_type="crash", start_age=44, duration_months=12),
    ],
)

result_crash = simulate(plan, market, policies, crash_sim)
print(f"Base case:  {result.success_probability:.1%}")
print(f"Crash at 44: {result_crash.success_probability:.1%}")
print(f"Impact: {result_crash.success_probability - result.success_probability:+.1%}")

## Sensitivity: What Matters Most?

In [ ]:
from monteplan.analytics.sensitivity import run_sensitivity

report = run_sensitivity(
    plan=plan, market=market, policies=policies,
    sim_config=SimulationConfig(n_paths=1000, seed=42),
    perturbation_pct=0.10,
)

top = sorted(report.results, key=lambda x: abs(x.impact), reverse=True)[:6]

fig, ax = plt.subplots(figsize=(9, 4))
names = [r.parameter_name for r in reversed(top)]
impacts = [r.impact for r in reversed(top)]
colors = ["#2ca02c" if i > 0 else "#d62728" for i in impacts]
ax.barh(range(len(names)), impacts, color=colors, alpha=0.7)
ax.set_yticks(range(len(names)))
ax.set_yticklabels(names)
ax.set_xlabel("Impact on Success Probability")
ax.set_title("FIRE Plan: What Matters Most?")
ax.axvline(0, color="black", linewidth=0.5)
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x:+.0%}"))
plt.tight_layout()
plt.show()

## Takeaways for Alex

1. **The base FIRE plan has moderate success** -- the 45-year horizon is challenging
2. **Spending is the biggest lever** -- reducing from $3,500 to $3,000/mo can dramatically improve success
3. **Delaying retirement even 3 years** (to 48) adds significant safety margin
4. **Guardrails spending** adapts better to long retirements than constant real
5. **A crash right before retirement** is the worst-case scenario -- consider a cash buffer
6. **Stock returns matter most** in the sensitivity analysis -- Alex's plan is equity-dependent